In [20]:
#!/usr/bin/env python
# coding: utf-8
get_ipython().run_line_magic('matplotlib', 'inline')
import pandas as pd
import time
import numpy as np
from datetime import datetime
from sklearn.externals import joblib 
import os
from konlpy.tag import Mecab
import lightgbm as lgb
print(lgb.__version__)

from sklearn import metrics

2.3.2


In [2]:
os.getcwd()

'/home/aiden/src/dacon_14'

In [3]:
base_path = '.'

df_train = pd.read_csv(os.path.join(base_path , 'input/train.csv'), index_col=0)
df_test = pd.read_csv(os.path.join(base_path , 'input/public_test.csv'), index_col=0)
df_test['smishing'] = -1

df_fea = pd.concat([df_train, df_test])
df_fea.shape

(297571, 3)

### Mecab

In [4]:
mecab = Mecab()
# df_space['morphs'] = df_space['spacing'].apply(lambda x: mecab.morphs(x))
df_fea['nouns'] = df_fea['text'].apply(lambda x: mecab.nouns(x))
# df_space['text_morphs'] = df_train['text'].apply(lambda x: mecab.morphs(x))

In [5]:
# from gensim.models import Word2Vec
# w2v_size = 100
# w2v_model = Word2Vec(df_fea['nouns'].values, 
#                            size=w2v_size, 
#                            window = 5, 
#                            min_count=25, 
#                            workers=16, 
#                            iter=100, 
#                            sg=1)
# ts = datetime.now().strftime('%Y%m%dT%H%M%S')
# file_name = 'model/gensim_w2v_{}'.format(ts)
# print(file_name)
# joblib.dump(w2v_model, '{}.pkl'.format(file_name))
# w2v_model.save('{}.model'.format(file_name))
# w2v_model

In [6]:
ts = '20191214T055747'
file_name = 'model/gensim_{}'.format(ts)

w2v_model = joblib.load(os.path.join(base_path, '{}.pkl'.format(file_name)))
w2v_size = w2v_model.wv.vectors.shape[1]

In [7]:
def mean_w2v(row):
    nouns = row['nouns']
    w2v = np.zeros(w2v_size)

    
    for n in nouns:
        if n in w2v_model.wv.vocab.keys():
            w2v = np.add(w2v, w2v_model.wv[n])
            
    return w2v if len(nouns) == 0 else np.true_divide(w2v, len(nouns))

w2v_cols = ['w2v_{}'.format(i) for i in range(w2v_size)]


df_fea[w2v_cols] = df_fea.apply(mean_w2v, axis=1, result_type='expand')

In [8]:
df_fea.head(10)

,year_month,text,smishing,nouns,w2v_0,w2v_1,w2v_2,w2v_3,w2v_4,w2v_5,...,w2v_90,w2v_91,w2v_92,w2v_93,w2v_94,w2v_95,w2v_96,w2v_97,w2v_98,w2v_99
id,,,,,,,,,,,,,,,,,,,,,
0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0,"[행성, 산, 팀장, 행복, 주말]",0.290056,-0.517329,-0.048548,0.035632,-0.246399,-0.081083,...,0.039810,0.155274,0.299007,0.084182,-0.106491,-0.144411,-0.081099,-0.011465,0.217347,0.479967
1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0,"[오늘, 하루, 시작, 은행, 진월동, 라운지]",0.163995,-0.247243,-0.090170,0.046783,0.004090,0.030764,...,-0.087980,0.106972,0.181236,0.032724,-0.220743,0.044146,-0.110678,0.199208,0.124589,0.353543
2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0,"[안녕, 고객, 은행, 금일, 납부, 금액, 원, 감사, 새해, 복, 은행, 옥포]",0.280096,-0.066877,-0.256147,-0.101468,-0.236363,0.075627,...,-0.385473,0.053096,0.090787,0.000227,-0.165890,-0.041340,-0.080808,0.046032,0.083723,0.275234
4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0,"[고객, 안녕, 은행, 지점, 해, 동안, 저희, 지점, 성원, 감사, 시작, 년,...",0.333083,-0.407860,-0.092240,0.046208,-0.211524,-0.027137,...,-0.417007,-0.095201,0.047777,-0.019826,-0.398968,-0.074437,-0.084755,0.063861,0.222251,0.202895
5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0,"[월, 한, 행복, 해]",0.297162,-0.326395,-0.022000,0.175641,-0.211487,0.025251,...,-0.487827,-0.010601,0.220540,0.168423,-0.214328,-0.132298,-0.294095,0.035665,0.262745,0.211478
6,2017-01,행복한주말보내세요XXX용현남전담직원대리 XXX올림,0,"[행복, 주말, 현남, 전담, 직원, 대리]",0.536358,-0.403108,0.146420,-0.056284,-0.241993,-0.025526,...,-0.400391,0.026811,0.213609,-0.140924,-0.324440,0.284893,0.255881,-0.052657,0.147549,0.480120
7,2017-01,XXX 고객님 안녕하세요XXX은행 무교지점 XXX과장입니다 오늘 아침에 눈을 뜨니 ...,0,"[고객, 안녕, 은행, 무교, 지점, 과장, 아침, 눈, 눈, 세상, 적, 눈, 눈...",0.309987,-0.396505,-0.056555,0.129030,-0.014572,-0.017500,...,-0.460663,0.012166,0.022573,-0.040356,-0.324134,-0.069774,-0.071868,0.178334,0.287738,0.274695
8,2017-01,XXX 고객님지난 한해 베풀어 주신 은혜 진심으로 감사 드립니다.가슴 깊이 간직 하...,0,"[고객, 한, 은혜, 진심, 감사, 가슴, 간직, 정유, 년, 새해, 가족, 행복,...",0.390770,-0.324953,-0.204903,-0.000504,-0.147635,0.067457,...,-0.369439,-0.005563,0.078228,-0.017127,-0.412095,-0.103872,-0.086155,0.055323,0.262111,0.360602
9,2017-01,설연휴 가족들과 훈훈한 정 나누시고 정겨운추억 많이 만드세요XXX오XXX올림,0,"[설, 연휴, 가족, 정, 추억, 오]",0.451833,-0.068016,-0.136574,0.029462,-0.433774,0.254634,...,-0.393862,-0.460240,0.038313,0.063135,-0.376351,0.116949,-0.211771,-0.005028,0.217925,0.225879


### tfidf

In [9]:
tfidf_size = 100
tfidf_cols = ['tfidf_{}'.format(i) for i in range(tfidf_size)]

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(max_features=tfidf_size)

df_tfidf = pd.DataFrame(vectorizer.fit_transform(df_fea['nouns'].apply(lambda x: ' '.join(x))).toarray(), columns=tfidf_cols)

### Merged

In [11]:
df_fea.shape, df_tfidf.shape

((297571, 104), (297571, 100))

In [12]:
df_merged = pd.concat([df_fea.reset_index(), df_tfidf[tfidf_cols]], axis=1)
df_merged.shape

(297571, 205)

In [13]:
df_merged.head(10)

,id,year_month,text,smishing,nouns,w2v_0,w2v_1,w2v_2,w2v_3,w2v_4,...,tfidf_90,tfidf_91,tfidf_92,tfidf_93,tfidf_94,tfidf_95,tfidf_96,tfidf_97,tfidf_98,tfidf_99
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0,"[행성, 산, 팀장, 행복, 주말]",0.290056,-0.517329,-0.048548,0.035632,-0.246399,...,0.0,0.000000,0.678718,0.000000,0.000000,0.000000,0.440442,0.0,0.000000,0.000000
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0,"[오늘, 하루, 시작, 은행, 진월동, 라운지]",0.163995,-0.247243,-0.090170,0.046783,0.004090,...,0.0,0.000000,0.000000,0.000000,0.435313,0.000000,0.000000,0.0,0.000000,0.000000
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0,"[안녕, 고객, 은행, 금일, 납부, 금액, 원, 감사, 새해, 복, 은행, 옥포]",0.280096,-0.066877,-0.256147,-0.101468,-0.236363,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0,"[고객, 안녕, 은행, 지점, 해, 동안, 저희, 지점, 성원, 감사, 시작, 년,...",0.333083,-0.407860,-0.092240,0.046208,-0.211524,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.323314,0.0,0.000000,0.000000
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0,"[월, 한, 행복, 해]",0.297162,-0.326395,-0.022000,0.175641,-0.211487,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000
5,6,2017-01,행복한주말보내세요XXX용현남전담직원대리 XXX올림,0,"[행복, 주말, 현남, 전담, 직원, 대리]",0.536358,-0.403108,0.146420,-0.056284,-0.241993,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.346004,0.0,0.000000,0.000000
6,7,2017-01,XXX 고객님 안녕하세요XXX은행 무교지점 XXX과장입니다 오늘 아침에 눈을 뜨니 ...,0,"[고객, 안녕, 은행, 무교, 지점, 과장, 아침, 눈, 눈, 세상, 적, 눈, 눈...",0.309987,-0.396505,-0.056555,0.129030,-0.014572,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
7,8,2017-01,XXX 고객님지난 한해 베풀어 주신 은혜 진심으로 감사 드립니다.가슴 깊이 간직 하...,0,"[고객, 한, 은혜, 진심, 감사, 가슴, 간직, 정유, 년, 새해, 가족, 행복,...",0.390770,-0.324953,-0.204903,-0.000504,-0.147635,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.575615,0.0,0.000000,0.000000
8,9,2017-01,설연휴 가족들과 훈훈한 정 나누시고 정겨운추억 많이 만드세요XXX오XXX올림,0,"[설, 연휴, 가족, 정, 추억, 오]",0.451833,-0.068016,-0.136574,0.029462,-0.433774,...,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
9,10,2017-01,(광고)XXXBaXXX고객님들 뒤엔XXX 언제나 XXX새로운 마음가짐으로 새롭게 준...,1,"[광고, 고객, 뒤, 마음가짐, 준비, 당, 행상, 품, 자격, 기준, 심사, 기준...",-0.020023,0.089636,-0.334222,-0.073732,-0.075779,...,0.0,0.126803,0.000000,0.062682,0.000000,0.368754,0.000000,0.0,0.065059,0.130257


In [14]:
# df_merged.to_csv('data_{}.csv'.format(ts))

In [15]:
cat_cols = []
fea_cols = w2v_cols + tfidf_cols + cat_cols

fea_cols

['w2v_0',
 'w2v_1',
 'w2v_2',
 'w2v_3',
 'w2v_4',
 'w2v_5',
 'w2v_6',
 'w2v_7',
 'w2v_8',
 'w2v_9',
 'w2v_10',
 'w2v_11',
 'w2v_12',
 'w2v_13',
 'w2v_14',
 'w2v_15',
 'w2v_16',
 'w2v_17',
 'w2v_18',
 'w2v_19',
 'w2v_20',
 'w2v_21',
 'w2v_22',
 'w2v_23',
 'w2v_24',
 'w2v_25',
 'w2v_26',
 'w2v_27',
 'w2v_28',
 'w2v_29',
 'w2v_30',
 'w2v_31',
 'w2v_32',
 'w2v_33',
 'w2v_34',
 'w2v_35',
 'w2v_36',
 'w2v_37',
 'w2v_38',
 'w2v_39',
 'w2v_40',
 'w2v_41',
 'w2v_42',
 'w2v_43',
 'w2v_44',
 'w2v_45',
 'w2v_46',
 'w2v_47',
 'w2v_48',
 'w2v_49',
 'w2v_50',
 'w2v_51',
 'w2v_52',
 'w2v_53',
 'w2v_54',
 'w2v_55',
 'w2v_56',
 'w2v_57',
 'w2v_58',
 'w2v_59',
 'w2v_60',
 'w2v_61',
 'w2v_62',
 'w2v_63',
 'w2v_64',
 'w2v_65',
 'w2v_66',
 'w2v_67',
 'w2v_68',
 'w2v_69',
 'w2v_70',
 'w2v_71',
 'w2v_72',
 'w2v_73',
 'w2v_74',
 'w2v_75',
 'w2v_76',
 'w2v_77',
 'w2v_78',
 'w2v_79',
 'w2v_80',
 'w2v_81',
 'w2v_82',
 'w2v_83',
 'w2v_84',
 'w2v_85',
 'w2v_86',
 'w2v_87',
 'w2v_88',
 'w2v_89',
 'w2v_90',
 'w2v_91'

In [16]:
train_size = len(df_train)
print(train_size)

295945


In [17]:
from sklearn.model_selection import train_test_split

model_index, hidden_index = train_test_split(range(train_size),     
    test_size=0.1, random_state=1984)

df_model = df_merged.iloc[model_index]
df_hidden = df_merged.iloc[hidden_index]
df_test = df_merged.loc[train_size:]

X = df_model[fea_cols].values
y = df_model['smishing'].values
print(np.unique(y, return_counts=True))

X_hidden = df_hidden[fea_cols].values
y_hidden = df_hidden['smishing'].values
print(np.unique(y_hidden, return_counts=True))


(array([0, 1]), array([249524,  16826]))
(array([0, 1]), array([27718,  1877]))


In [18]:
from sklearn.model_selection import StratifiedKFold

lgb_ts = datetime.now().strftime('%Y%m%dT%H%M%S')
[df_model.drop(c, axis=1, inplace=True) for c in df_model.columns if 'smishing_' in c]
[df_hidden.drop(c, axis=1, inplace=True) for c in df_hidden.columns if 'smishing_' in c]
[df_test.drop(c, axis=1, inplace=True) for c in df_test.columns if 'smishing_' in c]
skf = StratifiedKFold(n_splits=10)
print(skf)
cnt = 0
train_data = lgb.Dataset(X, label=y, feature_name=fea_cols, categorical_feature=cat_cols, free_raw_data=False)

StratifiedKFold(n_splits=10, random_state=None, shuffle=False)


In [19]:
initscore_filename = ''
param = {
    'boosting':'gbdt',
#     'boosting':'dart',
    'num_leaves': 7, 
    'objective': 'binary',
    'metric':'auc',
    'num_threads': 16,
    'learning_rate': 0.03,
    'is_unbalance': True,
    'bagging_fraction':0.02,
    'bagging_freq':1,
    'feature_fraction':0.01,
    'initscore_filename':initscore_filename,
    'lambda_l1':200,
    'lambda_l2':2000,
    'device_type':'gpu',
}
num_round = 10000

for train_index, valid_index in skf.split(X, y):
    cnt = cnt + 1
    print('\n', '*' * 20, cnt, '*' * 20)
    
    # init_bst_name = 'model.txt'
    # init_model = joblib.load(init_bst_name)
    # bst = None
    # init_model = bst
    train_set = train_data.subset(train_index).construct()
    valid_set = train_data.subset(valid_index).construct()
    print('train_set', np.unique(train_set.get_label(), return_counts=True))
    print('valid_set', np.unique(valid_set.get_label(), return_counts=True))
    bst = lgb.train(param, train_set, num_round, categorical_feature=cat_cols,
                    early_stopping_rounds=200, 
                    valid_sets=[train_set, valid_set],
                    verbose_eval=200,
                    # init_model=init_model,
                   )
    impt_dict = {k:v for k, v in zip(fea_cols, bst.feature_importance())}
    del_fea_cols = []
    print(sorted(impt_dict.items(), key=(lambda x:x[1]), reverse=True)[:5])
#     for r in sorted(impt_dict.items(), key=(lambda x:x[1]), reverse=True):
#         print(r)
#         if r[1] < 1:
#             del_fea_cols.append(r[0])
#     print(del_fea_cols)
#     print(len(del_fea_cols))
#     lgb.plot_importance(bst, height=0.3, figsize=(20, 100), max_num_features=100)
    df_model['smishing_{}'.format(cnt)] = bst.predict(X)        
    df_hidden['smishing_{}'.format(cnt)] = bst.predict(X_hidden)        
    df_test['smishing_{}'.format(cnt)] = bst.predict(df_test[fea_cols].values)    


 ******************** 1 ********************
train_set (array([0., 1.], dtype=float32), array([224571,  15143]))
valid_set (array([0., 1.], dtype=float32), array([24953,  1683]))
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.999525	valid_1's auc: 0.999562
[400]	training's auc: 0.999607	valid_1's auc: 0.999637
[600]	training's auc: 0.999615	valid_1's auc: 0.999645
Early stopping, best iteration is:
[441]	training's auc: 0.99962	valid_1's auc: 0.999652
[('w2v_2', 17), ('tfidf_13', 16), ('w2v_14', 15), ('w2v_56', 15), ('w2v_98', 15)]


/home/aiden/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/aiden/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/aiden/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pand


 ******************** 2 ********************
train_set (array([0., 1.], dtype=float32), array([224571,  15143]))
valid_set (array([0., 1.], dtype=float32), array([24953,  1683]))
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.999554	valid_1's auc: 0.999682
[400]	training's auc: 0.999624	valid_1's auc: 0.999729
[600]	training's auc: 0.999632	valid_1's auc: 0.999729
Early stopping, best iteration is:
[434]	training's auc: 0.999633	valid_1's auc: 0.999736
[('w2v_2', 16), ('w2v_9', 16), ('w2v_14', 15), ('w2v_7', 14), ('w2v_56', 14)]

 ******************** 3 ********************
train_set (array([0., 1.], dtype=float32), array([224571,  15143]))
valid_set (array([0., 1.], dtype=float32), array([24953,  1683]))
Training until validation scores don't improve for 200 rounds
[200]	training's auc: 0.999542	valid_1's auc: 0.999662
[400]	training's auc: 0.999618	valid_1's auc: 0.99972
Early stopping, best iteration is:
[390]	training's auc: 0.999623	valid_1'